In [342]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
warnings.filterwarnings('ignore')
from scipy import stats
from scipy.stats import norm, skew
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
data = pd.read_csv('Train2.csv')


#y_train = final_train.pop('LogPrice')
#y_test = final_test.pop('LogPrice')



In [343]:
## SalePrice as response


final_train = data.sample(frac=0.5, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]



#y_train = final_train.pop('SalePrice')
#y_test = final_test.pop('SalePrice')

In [344]:



y_train = final_train.pop('BoxCoxPrice')
y_test = final_test.pop('BoxCoxPrice')



In [345]:
import numpy
import pandas
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [346]:
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit

In [347]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

#http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor
#http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_regression.html

regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=9),random_state=0)
regr.fit(final_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=9, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=50, random_state=0)

In [348]:
from sklearn.model_selection import cross_val_score
#This uses cross validation to evaluate the score
cross_val_score(regr, final_train, y_train, cv=10)

array([ 0.99733237,  0.99972011,  0.94914739,  0.99915505,  0.99543158,
        0.99844488,  0.99369716,  0.99704496,  0.99867054,  0.99962393])

In [349]:
from sklearn.metrics import r2_score
print("Optimal nodes is {}".format(19))
regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=19),random_state=0)
print("Cross val score for {} nodes".format(19))
print(cross_val_score(regr, final_train, y_train, cv=10))

regr.fit(final_train, y_train)
y_1 = regr.predict(final_test)
accuracy = r2_score(y_test, y_1)
print("Cross-Predicted Accuracy: {}".format(accuracy))

Optimal nodes is 19
Cross val score for 19 nodes
[ 0.99733237  0.99972011  0.94914739  0.99915505  0.99543158  0.99844488
  0.99260007  0.99704496  0.99867054  0.99962393]
Cross-Predicted Accuracy: 0.9961935892945614


In [350]:
from QBUS2820 import rmse_jack, r2_jack 
#Jackknife method to be able to report the standard errors for the test results

columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Results']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, y_1)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, y_1))
results.round(3)

,Test RMSE,SE,Test R2,SE
Results,0.001,0.0,0.996,0.002


In [351]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=10)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)

In [352]:
#The decision tree with AdaBoosting
regr.fit(final_train, y_train)
pred_T = regr.predict(final_test)

In [353]:
#Combining the 2 predictions
pred_combined = (pred_L+pred_T)/2

In [354]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Lasso', 'Adaboost Tree', 'Lasso + Tree']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_L)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_L))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))

results.round(3)

,Test RMSE,SE,Test R2,SE
Lasso,0.001,0.0,0.996,0.001
Adaboost Tree,0.001,0.0,0.996,0.002
Lasso + Tree,0.001,0.0,0.998,0.001


In [355]:
#Elastic Net
from sklearn.linear_model import ElasticNetCV
enet = ElasticNetCV(l1_ratio=[0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99], cv=5)
enet.fit(final_train, np.ravel(y_train))

from sklearn.linear_model import ElasticNet
enet = ElasticNet(alpha=enet.alpha_, l1_ratio=enet.l1_ratio_)
enet.fit(final_train, np.ravel(y_train))

ElasticNet(alpha=7.8549277615487994e-05, copy_X=True, fit_intercept=True,
      l1_ratio=0.90000000000000002, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [356]:
#Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV

alphas = np.exp(np.linspace(-10,20,500)) 
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(final_train, np.ravel(y_train))
ridge = Ridge(alpha=ridge.alpha_)
ridge.fit(final_train, np.ravel(y_train))

Ridge(alpha=4.5399929762484854e-05, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [357]:
pred_R = ridge.predict(final_test)
pred_L = lasso.predict(final_test)
pred_E = enet.predict(final_test)

In [358]:
#Combining the all predictions
pred_combined = (pred_L+pred_T)/2
pred_combined2 = (pred_R+pred_T)/2
pred_combined3 = (pred_E+pred_T)/2
pred_combined4 = (pred_L+ + pred_R + pred_E + pred_T)/4

In [359]:
columns=['Test RMSE', 'SE', 'Test R2', 'SE']
rows = ['Lasso', 'Adaboost Tree', 'Lasso + Tree', 'Ridge', 'Enet', 'Tree + Ridge', 'Tree + Enet', 'Tree + All 3']
results=pd.DataFrame(0.0, columns=columns, index=rows) 

results.iloc[0,0], results.iloc[0,1] = rmse_jack(y_test, pred_L)
results.iloc[0,2], results.iloc[0,3] = (r2_jack(y_test, pred_L))
results.iloc[1,0], results.iloc[1,1] = rmse_jack(y_test, pred_T)
results.iloc[1,2], results.iloc[1,3] = (r2_jack(y_test, pred_T))
results.iloc[2,0], results.iloc[2,1] = rmse_jack(y_test, pred_combined)
results.iloc[2,2], results.iloc[2,3] = (r2_jack(y_test, pred_combined))
results.iloc[3,0], results.iloc[3,1] = rmse_jack(y_test, pred_R)
results.iloc[3,2], results.iloc[3,3] = (r2_jack(y_test, pred_R))
results.iloc[4,0], results.iloc[4,1] = rmse_jack(y_test, pred_E)
results.iloc[4,2], results.iloc[4,3] = (r2_jack(y_test, pred_E))
results.iloc[5,0], results.iloc[5,1] = rmse_jack(y_test, pred_combined2)
results.iloc[5,2], results.iloc[5,3] = (r2_jack(y_test, pred_combined2))
results.iloc[6,0], results.iloc[6,1] = rmse_jack(y_test, pred_combined3)
results.iloc[6,2], results.iloc[6,3] = (r2_jack(y_test, pred_combined3))
results.iloc[7,0], results.iloc[7,1] = rmse_jack(y_test, pred_combined4)
results.iloc[7,2], results.iloc[7,3] = (r2_jack(y_test, pred_combined4))
results.round(3)

,Test RMSE,SE,Test R2,SE
Lasso,0.001,0.0,0.996,0.001
Adaboost Tree,0.001,0.0,0.996,0.002
Lasso + Tree,0.001,0.0,0.998,0.001
Ridge,0.000,0.0,1.000,0.000
Enet,0.001,0.0,0.996,0.001
Tree + Ridge,0.001,0.0,0.999,0.001
Tree + Enet,0.001,0.0,0.998,0.001
Tree + All 3,0.001,0.0,0.999,0.000
